In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('checkins_clean.csv',sep=';')
data.head()


,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 18.2+ MB


In [4]:
from sklearn.cluster import MeanShift
model=MeanShift(bandwidth=0.1)
data_part=data.iloc[0:100000,:]
data_part.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [5]:
model.fit(data_part.loc[:,('latitude','longitude')])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [6]:
data_part.loc[:,('latitude','longitude')].head(10)

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037
5,32.221743,-110.926479
6,40.650000,-73.950000
7,33.448377,-112.074037
8,33.414768,-111.909309
9,42.358431,-71.059773


In [7]:
labels = model.labels_
len(np.unique(labels))

3231

In [8]:
cluster_centers = model.cluster_centers_
cluster_centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [9]:
(unique, counts) = np.unique(labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
k=pd.DataFrame(frequencies)


In [51]:
clusters=k[k[1]>15][0].values
len(clusters)

592

In [52]:
cluster_centers[clusters]

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [  41.61853175,  -88.44556818],
       [  39.2494686 ,  -77.1821271 ],
       [  38.65877915,  -76.8856871 ]])

In [53]:
offices=np.array([[33.751277, -118.188740],
                 [25.867736, -80.324116],[51.503016, -0.075479], [52.378894, 4.885084], [39.366487, 117.036146],
                 [-33.868457, 151.205134]] )

In [54]:
offices

array([[ 3.37512770e+01, -1.18188740e+02],
       [ 2.58677360e+01, -8.03241160e+01],
       [ 5.15030160e+01, -7.54790000e-02],
       [ 5.23788940e+01,  4.88508400e+00],
       [ 3.93664870e+01,  1.17036146e+02],
       [-3.38684570e+01,  1.51205134e+02]])

In [55]:
from scipy.spatial import distance
dist_matr=distance.cdist(cluster_centers[clusters], offices, 'euclidean')
dist_matr

array([[ 44.74257092,  16.14371999,  74.69906582,  79.73425538,
        191.0327603 , 237.22725876],
       [  6.19395917,  32.5726786 , 113.37331719, 118.41008135,
        229.11470442, 271.67953165],
       [  6.29424146,  32.47447468, 113.2748328 , 118.31160891,
        229.01456477, 271.58164907],
       ...,
       [ 30.76605219,  17.72133043,  88.92117683,  93.94890117,
        205.49405482, 251.2583223 ],
       [ 41.37357142,  13.74564883,  78.07422497,  83.1108233 ,
        194.21830835, 239.80611351],
       [ 41.5935783 ,  13.24513413,  77.87671339,  82.91381403,
        193.92312447, 239.34415107]])

In [56]:
np.argmin(dist_matr)

2507

In [57]:
ind = np.unravel_index(np.argmin(dist_matr, axis=None), dist_matr.shape)
ind
#(0, 0)
#>>> a[ind]
#10

(417, 5)

In [74]:
cluster_centers[58]

array([51.50299126, -0.12553729])

In [75]:
dist_matr[58]

array([1.19390298e+02, 8.41960708e+01, 5.00582948e-02, 5.08660312e+00,
       1.17788602e+02, 1.73750558e+02])

In [76]:
offices[2]

array([51.503016, -0.075479])

In [62]:
np.shape(dist_matr)

(592, 6)

In [63]:
def get_indices_of_k_smallest(arr, k):
    idx = np.argpartition(arr.ravel(), k)
    return tuple(np.array(np.unravel_index(idx, arr.shape))[:, range(min(k, 0), max(k, 0))])
indices = get_indices_of_k_smallest(dist_matr, 4)
indices

(array([417, 369, 416,  58], dtype=int64), array([5, 3, 1, 2], dtype=int64))

In [88]:
#считаем расстояние по частям, так как на общем массиве получается фигня
test=distance.cdist(cluster_centers[50:600], offices, 'euclidean')
get_indices_of_k_smallest(test, 4)

(array([370, 320, 369,   8], dtype=int64), array([5, 3, 1, 2], dtype=int64))

In [90]:
cluster_centers[420]

array([-33.86063043, 151.20477593])

In [78]:
str(cluster_centers[58][0])+' '+'1'

'51.502991260887086 1'

In [91]:
file=open('subc3w11.txt','w+')
file.write(str(cluster_centers[420][0])+' '+str(cluster_centers[420][1]))
file.close()

In [92]:
pip install artm


The following command must be run outside of the IPython shell:

    $ pip install artm

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/
